# Bot from GPT to **Telegram**

This notebook is the final project for the NLP Сourse at GeekBrains, Faculty of Artificial Intelligence.

The notebook contains code for implementing the bot, which is a dialog script. The bot was trained on a dataset of compliments from the Internet, so the ability to receive a random compliment was added at the user’s request.

In [ ]:
#!pip install transformers
#!pip install python-telegram-bot==13.7

In [ ]:
#!wget https://gist.githubusercontent.com/vddavydova/87269a661fc992328a3658468e06d8d1/raw/8befb1e716b84d1cef2397e0c29feb62ebb973ad/compliments.txt

In [ ]:
import random
import pathlib

from telegram import Update, ReplyKeyboardMarkup, ReplyKeyboardRemove
from telegram import InlineKeyboardMarkup, InlineKeyboardButton
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, CallbackQueryHandler

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# loading training data
with open('compliments.txt', 'r') as f:
    compliments = f.read().strip().split('\n')
print(len(compliments))
for i in range(3):
    print(compliments[i])

325
Ты так красива, что я просто не могу отвести взгляд.
Твои большие глаза делают тебя такой милой.
Ты всегда выглядишь так свежо!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# loading token for bot
data_path = pathlib.Path('/content/drive/MyDrive/Colab Notebooks/GPT Week Yandex/data/token-bot.txt')

token = [
    l for l in open(data_path, 'r').read().split('\n')
    if len(l.strip()) > 0
]

token = str(token).strip("[]'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loading compliments from a file
with open('compliments.txt', 'r') as f:
    compliments = f.read().strip().split('\n')

# tokenizer and model initialization
model_name = 'sberbank-ai/rugpt3large_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [ ]:
# generating bot responses
def generate_response(text):
    prefix = f'\nx:{text}\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size + 128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [ ]:
# generation of compliments
def generate_compliment():
    return random.choice(compliments)

In [ ]:
# select a bot response option
def respond_to_dialog(update: Update, context: CallbackContext) -> None:
    user_input = update.message.text

    if "I want a compliment" in user_input:
        compliment = generate_compliment()
        #update.message.reply_text(compliment, reply_markup=ReplyKeyboardRemove())
        update.message.reply_text(compliment)

    else:
        # if not, generate a response using the existing logic
        result = generate_response(user_input)
        update.message.reply_text(result)

In [ ]:
# bot implementation
def start(update: Update, context: CallbackContext) -> None:
    user_id = update.message.from_user.id
    keyboard = [['I want a compliment']]
    update.message.reply_text('Welcome! Start a dialogue with the bot with any phrase', reply_markup=ReplyKeyboardMarkup(keyboard))

def main() -> None:
    updater = Updater(token)

    dp = updater.dispatcher

    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(MessageHandler(Filters.text & ~Filters.command, respond_to_dialog))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()